In [1]:
import numpy as np
from figaro.utils import rejection_sampler
from figaro.cosmology import Planck18 as Omega, dVdz_approx_planck18
import matplotlib.pyplot as plt
import os

In [2]:
true_H0 = 70. # km/s/Mpc

In [3]:
# Mass distribution
from population_models.mass import plpeak

# Luminoisty distance distribution
dL_max = 5000
def dLsq(dL):
    return 3*dL**2/dL_max**3

In [4]:
# Generate true detector frame mass samples
def generate_true_mz(n_SE_draws):
    M  = rejection_sampler(n_SE_draws, plpeak, [0,200])
    dL = rejection_sampler(n_SE_draws, dLsq, [0,dL_max])
    z = Omega.Redshift(dL)
    return np.array([M * (1 + z), dL]).T

In [5]:
# Generate detector frame mass posterior samples
def generate_posterior_samples(truth, sigma = 0.1, n_samples = 1000):
    log_obs_mean = np.log(truth) + np.random.normal(0, sigma, truth.shape) # shape = (n_events, n_params)
    return np.exp(np.random.normal(log_obs_mean, sigma, (n_samples, truth.shape[0], truth.shape[1]))).transpose(1,2,0) # shape = (n_events, n_params, n_samples)

In [6]:
from selection_function import selection_function

n_SE_draws = 5000
outdir = 'hierarchical_SE_test_5'
if not os.path.exists(outdir):
    os.makedirs(outdir)

samples = generate_true_mz(n_SE_draws)
posterior = generate_posterior_samples(samples[np.random.uniform(0,1,samples.shape[0]) <= selection_function(samples)])

samples.shape, posterior.shape

/Users/thomas.ng/Documents/GitHub/pop_models_lvk/population_models/mass.py:29: RuntimeWarning: divide by zero encountered in power
  p = m**-alpha * (alpha-1.)/(mmin**(1.-alpha)-mmax**(1.-alpha))
/Users/thomas.ng/Documents/GitHub/pop_models_lvk/population_models/mass.py:10: RuntimeWarning: overflow encountered in exp
  p[idx]       = 1./(1.+np.exp(exponent))


((5000, 2), (152, 2, 1000))

In [7]:
np.savetxt(outdir+'/true_samples.txt', samples)

In [8]:
posterior.max(axis=(0,2))

array([ 147.13318334, 8670.41355312])

In [9]:
np.savetxt(outdir+'/jsd_bounds.txt', np.array([np.median(posterior, axis=2).min(axis=0), np.median(posterior, axis=2).max(axis=0)]).transpose())

In [10]:
if not os.path.exists(outdir+'/data'):
    os.makedirs(outdir+'/data')
for i,p in enumerate(posterior):
    np.savetxt(f'{outdir}/data/posterior_samples_{i+1}.txt', p.T)